# Analyzing Motorcycle Parts Sales Data

In this project, we will be analyzing the sales data of a company that sells motorcycle parts. They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. Their `sales` table includes the following data:


## Sales
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |

We will run a query to get an overview of the data in detail:

In [1]:
SELECT *
FROM sales
LIMIT 10;

,order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
0,N1,2021-06-01 00:00:00+00:00,North,Retail,Braking system,9,19.29,173.61,Cash,0.00
1,N2,2021-06-01 00:00:00+00:00,North,Retail,Suspension & traction,8,32.93,263.45,Credit card,0.03
2,N3,2021-06-01 00:00:00+00:00,North,Wholesale,Frame & body,16,37.84,605.44,Transfer,0.01
3,N4,2021-06-01 00:00:00+00:00,North,Wholesale,Suspension & traction,40,37.37,1494.80,Transfer,0.01
4,N5,2021-06-01 00:00:00+00:00,North,Retail,Frame & body,6,45.44,272.61,Credit card,0.03
5,N6,2021-06-02 00:00:00+00:00,North,Retail,Frame & body,1,40.41,40.41,Credit card,0.03
6,N7,2021-06-02 00:00:00+00:00,North,Retail,Miscellaneous,6,20.28,121.66,Credit card,0.03
7,N8,2021-06-03 00:00:00+00:00,North,Retail,Electrical system,9,20.50,184.54,Credit card,0.03
8,N9,2021-06-03 00:00:00+00:00,North,Retail,Suspension & traction,5,36.18,180.91,Credit card,0.03
9,N10,2021-06-03 00:00:00+00:00,North,Retail,Electrical system,5,28.33,141.67,Credit card,0.03


Now, we've been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

In [6]:
SELECT
    CASE WHEN EXTRACT('month' from date) = 6 THEN 'June'
        WHEN EXTRACT('month' from date) = 7 THEN 'July'
        WHEN EXTRACT('month' from date) = 8 THEN 'August'
    END as month,
    warehouse,
	product_line,
	SUM(total) - SUM(payment_fee) AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY month, warehouse, product_line
ORDER BY month, product_line, net_revenue DESC


,month,warehouse,product_line,net_revenue
0,August,Central,Braking system,3039.41
1,August,West,Braking system,2500.67
2,August,North,Braking system,1770.84
3,August,North,Electrical system,4721.12
4,August,Central,Electrical system,3126.43
5,August,West,Electrical system,1241.84
6,August,Central,Engine,9528.71
7,August,North,Engine,2324.19
8,August,Central,Frame & body,8657.99
9,August,North,Frame & body,7898.89


The finance team is exploring ways to reduce transaction costs and improve profitability. Now they want to determine the most profitable payment method for each warehouse in each month. 

Our task here is to calculate the net revenue for each payment method, grouped by warehouse and month, and identify the top payment method for each combination.

In [7]:
WITH monthly_revenue AS (
    SELECT
        CASE WHEN EXTRACT('month' from date) = 6 THEN 'June'
            WHEN EXTRACT('month' from date) = 7 THEN 'July'
            WHEN EXTRACT('month' from date) = 8 THEN 'August'
        END as month,
        warehouse,
        payment,
        SUM(total) - SUM(payment_fee) AS net_revenue
    FROM sales
    GROUP BY month, warehouse, payment
),

ranked_revenue AS (
	SELECT
		month,
		warehouse,
		payment,
		net_revenue,
		RANK() OVER(PARTITION BY month, warehouse ORDER BY net_revenue DESC) AS revenue_rank
	FROM monthly_revenue
)

SELECT
	month,
	warehouse,
	payment,
	net_revenue
FROM ranked_revenue
WHERE revenue_rank = 1
ORDER BY month, warehouse, net_revenue DESC
;

,month,warehouse,payment,net_revenue
0,August,Central,Transfer,31509.00
1,August,North,Transfer,23480.13
2,August,West,Transfer,6466.42
3,July,Central,Transfer,23893.59
4,July,North,Transfer,17585.25
5,July,West,Transfer,7606.51
6,June,Central,Transfer,23453.08
7,June,North,Transfer,17000.12
8,June,West,Transfer,8645.98


The marketing team is planning a targeted campaign and wants to know the most popular product lines for retail and wholesale customers. 

Our task is to find the top 3 most ordered product lines for each client type.

In [8]:
WITH ranked_sales AS (
	SELECT
		client_type,
		product_line,
		SUM(quantity) AS total_quantity_ordered,
		ROW_NUMBER() OVER (PARTITION BY client_type ORDER BY SUM(quantity) DESC) AS rank
	FROM sales
	GROUP BY client_type, product_line
)

SELECT
	client_type,
	product_line,
	total_quantity_ordered,
	rank
FROM ranked_sales
WHERE rank <= 3
ORDER BY client_type, rank;

,client_type,product_line,total_quantity_ordered,rank
0,Retail,Suspension & traction,1017,1
1,Retail,Electrical system,862,2
2,Retail,Braking system,862,3
3,Wholesale,Braking system,1268,1
4,Wholesale,Suspension & traction,1128,2
5,Wholesale,Frame & body,932,3
